# Сложные объединения
---

Вы уже умеете присоединять строки друг к другу путём добавления столбцов «сбоку» — с помощью различных видов `JOIN`.

А что если нам необходимо присоединить несколько результатов «снизу», так, чтобы получить общий результат в выводе?

Ответу на этот вопрос и посвящён текущий модуль.

## (1) Знакомство с данными
---

В данном блоке мы будем работать с данными о компании, организующей перевозки грузов.

Интересующие нас данные хранятся в таблицах `city`, `customer`, `driver`, `shipment`, `truck`. Давайте внимательно их изучим.

Ниже представлена *ER*-диаграмма (от англ. *entity-relation*, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.

![](data/dst3-u2-md4_1_1.jpg)

Таблица `city` — это справочник городов. Структура справочника представлена ниже.

| НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                    ОПИСАНИЕ                     |
|:-------------:|:----------:|:-----------------------------------------------:|
|    city_id    |  integer   | уникальный идентификатор города, первичный ключ |
|   city_name   |    text    |                 название города                 |
|     state     |    text    |        штат, к которому относится город         |
|  population   |  integer   |                население города                 |
|     area      |  numeric   |                 площадь города                  |

Таблица `customer` — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:

| НАЗВАНИЕ ПОЛЯ  | ТИП ДАННЫХ |                     ОПИСАНИЕ                      |
|:--------------:|:----------:|:-------------------------------------------------:|
|    cust_id     |  integer   | уникальный идентификатор клиента, первичный ключ  |
|   cust_name    |    text    |                 название клиента                  |
| annual_revenue |  numeric   |                 ежегодная выручка                 |
|   cust_type    |    text    |                 тип пользователя                  |
|    address     |    text    |                       адрес                       |
|      zip       |  integer   |                  почтовый индекс                  |
|     phone      |    text    |                      телефон                      |
|    city_id     |  integer   | идентификатор города, внешний ключ к таблице city |

Следующая таблица — `driver` — справочник водителей. Перечень сведений, содержащихся в таблице, представлен ниже.

| НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                          ОПИСАНИЕ                          |
|:-------------:|:----------:|:----------------------------------------------------------:|
|   driver_id   |  integer   |     уникальный идентификатор водителя, первичный ключ      |
|  first_name   |    text    |                        имя водителя                        |
|   last_name   |    text    |                      фамилия водителя                      |
|    address    |    text    |                       адрес водителя                       |
|   zip_code    |  integer   |                  почтовый индекс водителя                  |
|     phone     |    text    |                      телефон водителя                      |
|    city_id    |  integer   | идентификатор города водителя, внешний ключ к таблице city |

В таблице `truck` хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:

| НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                      ОПИСАНИЕ                      |
|:-------------:|:----------:|:--------------------------------------------------:|
|   truck_id    |  integer   | Уникальный идентификатор грузовика, первичный ключ |
|     make      |    text    |              Производитель грузовика               |
|  model_year   |  integer   |               Дата выпуска грузовика               |

Последняя таблица в датасете, `shipment`, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.

| НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                                       ОПИСАНИЕ                                        |
|:-------------:|:----------:|:-------------------------------------------------------------------------------------:|
|    ship_id    |  integer   |                   уникальный идентификатор доставки, первичный ключ                   |
|    cust_id    |  integer   | идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer  |
|    weight     |  numeric   |                                      вес посылки                                      |
|   truck_id    |  integer   | идентификатор грузовика, на котором отправлена доставка, внешний ключ к таблице truck |
|   driver_id   |  integer   |  идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver  |
|    city_id    |  integer   |    идентификатор города в который совершена доставка, внешний ключ к таблице city     |
|   ship_date   |    date    |                                     дата доставки                                     |

### Задание 1.1

> Укажите название города с максимальным весом единичной доставки.


```sql
   SELECT city.city_name
     FROM SQL.shipment AS shipment
     JOIN SQL.city AS city ON shipment.city_id = city.city_id
 GROUP BY city.city_id,
          shipment.weight
 ORDER BY shipment.weight DESC
    LIMIT 1
```

Вариант после повторения


```sql
   SELECT city.city_name
     FROM SQL.shipment AS shipment
     JOIN SQL.city AS city ON shipment.city_id = city.city_id
 ORDER BY shipment.weight DESC
    LIMIT 1
```

**Ответ:** Green Bay



> Сколько различных производителей грузовиков перечислено в таблице `truck`?


```SQL
   SELECT COUNT(DISTINCT truck.make)
     FROM SQL.truck AS truck
```

**Ответ:** 3

> Как зовут водителя (`first_name`), который совершил наибольшее количество доставок одному клиенту?


```sql
   SELECT concat (driver.first_name, ' ', driver.last_name)
     FROM SQL.shipment AS shipment
     JOIN SQL.driver AS driver ON shipment.driver_id = driver.driver_id
 GROUP BY shipment.cust_id,
          shipment.driver_id,
          driver.first_name,
          driver.last_name
 ORDER BY COUNT(shipment.driver_id) DESC
    LIMIT 1
```

**Ответ:** Holger (Nohr)

> Укажите даты первой и последней по времени доставок в таблице `shipment`.
>
> Ответ введите в формате ДД.ММ.ГГГГ.


```sql
   SELECT shipment.ship_date
     FROM SQL.shipment AS shipment
 ORDER BY shipment.ship_date ASC
    LIMIT 1
```

Вариант после повторения

```sql
   SELECT concat (
          to_char (MIN(shipment.ship_date), 'DD.MM.YYYY'),
          ' ',
          to_char (MAX(shipment.ship_date), 'DD.MM.YYYY')
          ) AS shipments_period
     FROM SQL.shipment AS shipment
```

Ответ: 08.01.2016 27.12.2017

> Укажите имя клиента, получившего наибольшее количество доставок за 2017 год.


```sql
   SELECT customer.cust_name
     FROM SQL.shipment AS shipment
     JOIN SQL.customer AS customer ON shipment.cust_id = customer.cust_id
 GROUP BY shipment.cust_id,
          customer.cust_name
 ORDER BY COUNT(shipment.cust_id) DESC
    LIMIT 1
```

**Ответ:** Autoware In

## (2-6) UNION

---

Вернёмся к центральному вопросу модуля: как соединить несколько результатов, чтобы получить в выводе один общий?

### (2) UNION. Принцип и условия работы.
---

Чтобы разобраться в этом вопросе, смоделируем ситуацию.

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем простой запрос:


```sql
   SELECT book_name object_name,
          'книга' object_description
     FROM public.books
UNION ALL
   SELECT movie_title,
          'фильм'
     FROM SQL.kinopoisk
```

Визуально произведённое нами действие можно представить следующим образом:

![](data/dst3-u2-md4_2_1.png)

Общий принцип мы поняли, разберёмся в деталях:

В запросе мы использовали оператор UNION ALL — он присоединяет любой результат запроса к другому «снизу» при условии, что у них **одинаковая структура**, а именно:

* одинаковый тип данных;

    ![](data/dst3-u2-md4_2_2.png)

* одинаковое количество столбцов;

    ![](data/dst3-u2-md4_2_3.png)

* одинаковый порядок столбцов согласно типу данных.

    ![](data/dst3-u2-md4_2_4.png)

**Виды `UNION`**

Оператор присоединения существует в двух вариантах:

* `UNION` выводит только уникальные записи;
* `UNION ALL` присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.

**Важно!** `UNION` оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с `DISTINCT`). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать `UNION ALL`.


**Синтаксис**


```SQL
   SELECT n columns
     FROM table_1
UNION ALL
   SELECT n columns
     FROM table_2 -- ... 
UNION ALL
   SELECT n columns
     FROM table_n
```

Пришла пора испытать функцию `UNION ALL`  на практике.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с ID всех таблиц и указанием объекта, к которому он относится.


```sql
   SELECT c.city_id object_name,
          'id города' object_type /*выбираем колонку city_id и задаём ей алиас object_name, сами задаём объект 'id города' и название столбца object_type*/
     FROM SQL.city c /*из схемы sql и таблицы city, задаём алиас таблице — с*/
UNION ALL
/*оператор присоединения*/
   SELECT d.driver_id other_name,
          'id водителя' other_type /*выбираем колонку driver_id и задаём ей алиас other_name, сами задаём объект 'id водителя' и название столбца other_type*/
     FROM SQL.driver d /*из схемы sql и таблицы driver, задаём алиас таблице — d*/
UNION ALL
/*оператор присоединения*/
   SELECT s.ship_id,
          'id доставки' /*выбираем колонку ship_id, сами задаём объект 'id доставки'*/
     FROM SQL.shipment s /*из схемы sql и таблицы shipment, задаём алиас таблице — s*/
UNION ALL
/*оператор присоединения*/
   SELECT c.cust_id,
          'id клиента' /*выбираем колонку cust_id, сами задаём объект 'id клиента'*/
     FROM SQL.customer c /*из схемы sql и таблицы customer, задаём алиас таблице — c*/
UNION ALL
/*оператор присоединения*/
   SELECT t.truck_id,
          'id грузовика' /*выбираем колонку truck_id, сами задаём объект 'id грузовика'*/
     FROM SQL.truck t /*из схемы sql и таблицы truck, задаём алиас таблице — t*/
 ORDER BY 1 /*сортировка по первому столбцу*/
```

**Обратите внимание!** Несмотря на исходные названия колонок *other_name* и *other_type* во втором подзапросе, в выводе мы получим названия, которые дали в первом блоке: *object_name* и *object_type*.

Другая особенность — в применении сортировки `ORDER BY`: она всегда будет относиться к итоговому результату всего запроса с `UNION ALL`.

В случаях, когда необходимо применить команду `ORDER BY` или `LIMIT` не к итоговому результату, а к каждой части запроса, можно обернуть подзапросы в скобки.

Чтобы посмотреть, как это работает, вернёмся к нашему примеру с общим справочником по фильмам и книгам.

Мы уже знаем, что можно легко и непринуждённо применить операторы `ORDER BY` и `LIMIT` ко всему результату запроса.


```sql
   SELECT book_name object_name,
          'книга' object_descritption
     FROM public.books
UNION ALL
   SELECT movie_title,
          'фильм'
     FROM SQL.kinopoisk
 ORDER BY 1
    LIMIT 1
```

Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.

А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.

Нет ничего проще — отсортируем каждую часть запроса по отдельности и объединим результаты!

Просто добавим ORDER BY и LIMIT ещё и в первую часть запроса:


```sql
   SELECT book_name object_name,
          'книга' object_descritption
     FROM public.books
 ORDER BY 1
    LIMIT 1
UNION ALL
   SELECT movie_title,
          'фильм'
     FROM SQL.kinopoisk
 ORDER BY 1
    LIMIT 1
```


Вместо результата получим сообщение о синтаксической ошибке: *"...syntax error at or near "UNION"..."* Очевидно, этот фокус не удался.

Не стоит огорчаться, ведь проблему можно решить одним (ну, почти) движением руки — просто добавив скобки вокруг каждой из частей запроса.


```sql
(
   SELECT book_name object_name,
          'книга' object_descritption
     FROM public.books
 ORDER BY 1
    LIMIT 1
)
UNION ALL
(
   SELECT movie_title,
          'фильм'
     FROM SQL.kinopoisk
 ORDER BY 1
    LIMIT 1
)
```

#### Задание 2.1

> Напишите запрос, который создает уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков. Результатом запроса должны быть два столбца: название и тип объекта (`city`, `state`, `driver`, `truck`). Отсортируйте список по названию объекта, а затем — по типу.

```sql
   SELECT DISTINCT city.city_name AS object_name,
          'city' AS object_type
     FROM SQL.city AS city
UNION ALL
   SELECT DISTINCT city.state,
          'state'
     FROM SQL.city AS city
UNION ALL
   SELECT DISTINCT driver.first_name,
          'driver'
     FROM SQL.driver AS driver
UNION ALL
   SELECT DISTINCT truck.make,
          'truck'
     FROM SQL.truck AS truck
 ORDER BY object_name,
          object_type
```

**Ошибка в формулировке:** не уникальный справочник, а справочник уникальных значений из таблиц:

**Эталонное решение**


```sql
   SELECT c.city_name "название",
          'city' "тип объекта"
     FROM SQL.city c
    UNION
   SELECT c.state,
          'state'
     FROM SQL.city c
    UNION
   SELECT d.first_name,
          'driver'
     FROM SQL.driver d
    UNION
   SELECT t.make,
          'truck'
     FROM SQL.truck t
 ORDER BY 1,
          2
```

#### Задание 2.2

> Напишите запрос, который соберёт имена всех упомянутых городов и штатов из таблицы `city`. Результатом запроса должен быть один столбец `object_name`, отсортированный в алфавитном порядке.


```sql
   SELECT city_name AS object_name
     FROM SQL.city
UNION ALL
   SELECT state
     FROM SQL.city
 ORDER BY object_name
```

#### Задание 2.3

> Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов. Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов. Результатом запроса должен быть один столбец `object_name`, отсортированный в алфавитном порядке.


```SQL
   SELECT DISTINCT cities_states.object_name
     FROM (
             SELECT city_name AS object_name
               FROM SQL.city
          UNION ALL
             SELECT state
               FROM SQL.city
           ORDER BY object_name
          ) AS cities_states
```

**!!Эталонное решение!!**


```sql
   SELECT c.city_name object_name
     FROM SQL.city c
    UNION
   SELECT c.state
     FROM SQL.city c
 ORDER BY 1
```

### (3) UINION и ограничение типов данных
---

**Почему так важен тип данных?**

Как мы уже знаем, `UNION` может быть использован только в случае полного соответствия столбцов и их типов в объединяемых запросах.

Допустим, мы хотим вывести список всех `id` городов и их названий в одном столбце.

Давайте напишем запрос, который позволит получить нужный нам результат.


```sql
   SELECT c.city_id
     FROM SQL.city c
UNION ALL
   SELECT cc.city_name
     FROM SQL.city cc
```

Вместо результата вы получите сообщение об ошибке: *"ERROR: UNION types integer and text cannot be matched"*. Дело в том, что мы попытались объединить числовой и 
строковый типы в одной колонке, а это невозможно.

Если мы всё же хотим выполнить поставленную задачу, придётся привести оба столбца к одному типу данных. Не каждый текст может быть приведён к числу, зато каждое число может быть представлено в текстовом формате.

Забегая вперёд, скажем пару слов о типизации столбцов. Для типизации в `Postgres` составляется запрос по модели `column_name::column_type`.

Таким образом, чтобы перевести `city_id` в текст, нам потребуется написать `city_id::text`.

**Важно!** Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.


Немного подправим запрос, чтобы получить желаемый результат.


```sql
   SELECT c.city_id::text
     FROM sql.city c
UNION ALL
   SELECT cc.city_name
     FROM sql.city cc
```

#### Задание 3.1

> Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник `contact`. Также добавьте столбец с именем водителя `first_name` и столбец `contact_type` с типом контакта (`phone` или `zip` в зависимости от типа). Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.


```SQL
   SELECT phone AS contact,
          first_name,
          'phone' AS contact_type
     FROM sql.driver
UNION ALL
   SELECT zip_code::text,
          first_name,
          'zip'
     FROM sql.driver
 ORDER BY contact,
          first_name
```

**Эталонное решение**


```sql
   SELECT d.zip_code::text contact,
          d.first_name first_name,
          'zip' contact_type
     FROM sql.driver d
    UNION
   SELECT dd.phone contact,
          dd.first_name first_name,
          'phone' contact_type
     FROM sql.driver dd
 ORDER BY 1,
          2
```

### (4) UNION ALL и промежуточные итоги
---

**Возможности UNION**

Помимо соединения разнородных сущностей в единый справочник, `UNION ALL` часто используется для подведения промежуточных итогов и выведения результатов агрегатных функций.

Кроме агрегатных функций, в запросах с `UNION` могут использоваться функции группировки и выборки.


```sql
   SELECT c.city_name,
          c.population
     FROM sql.city c
UNION ALL
   SELECT 'total',
          SUM(c.population)
     FROM sql.city c
 ORDER BY 2 DESC
```

Визуально это действие можно представить так:

![](data/dst3-u2-md4_4_1.png)

Выполнив запрос в Metabase, вы заметите, что сначала выведен итоговый результат с суммой населения всех городов в таблице, а затем по каждому городу в отдельности.

#### Задание 4.1

> Напишите запрос, который выводит общее число доставок `total_shipments`, а также количество доставок в каждый день. Необходимые столбцы: `date_period`, `cnt_shipment`. Не забывайте о единой типизации. Упорядочите по убыванию столбца `date_period`.


```sql
   SELECT dates_cnt.date_period::text,
          dates_cnt.cnt_shipment
     FROM (
             SELECT ship_date AS date_period,
                    COUNT(ship_id) AS cnt_shipment
               FROM sql.shipment
           GROUP BY ship_date
           ORDER BY date_period DESC
          ) AS dates_cnt
UNION ALL
   SELECT 'total',
          COUNT(ship_id)
     FROM sql.shipment
```


**Решение после повторения**

```sql
   SELECT s.ship_date::text AS date_period,
          count(s.ship_id) AS cnt_shipment
     FROM sql.shipment AS s
 GROUP BY s.ship_date
UNION ALL
   SELECT 'total_shipments',
          count(s.ship_id)
     FROM sql.shipment AS s
 ORDER BY 1 DESC
```

**Эталонное решение**

```sql
   SELECT s.ship_date::text date_period,
          COUNT(*) cnt_shipment
     FROM sql.shipment s
 GROUP BY 1
UNION ALL
   SELECT 'total_shipments',
          COUNT(*)
     FROM sql.shipment s
 ORDER BY 1 DESC
```

### (5) UNION и дополнительные условия
---

`UNION` также может быть использован для разделения существующей выборки по критерию «выполнение определённого условия».

Например, с помощью `UNION` можно отобразить, у кого из водителей заполнен столбец с номером телефона.


```sql
   SELECT d.first_name,
          d.last_name,
          'телефон заполнен' phone_info
     FROM sql.driver d
    WHERE d.phone IS NOT NULL
    UNION
   SELECT d.first_name,
          d.last_name,
          'телефон не заполнен' phone_info
     FROM sql.driver d
    WHERE d.phone IS NULL
```

#### Задание 5.1

> Напишите запрос, который выведет все города и штаты, в которых они расположены, а также информацию о том, была ли осуществлена доставка в этот город:
>
> * если в город была осуществлена доставка, то выводим 'доставка осуществлялась';
> * если нет — выводим 'доставка не осуществлялась'.
>
> Столбцы к выводу: `city_name`, `state`, `shipping_status`. Отсортируйте в алфавитном порядке по городу, а затем — по штату.

**Решение после повторения**

```sql
   SELECT c.city_name,
          c.state,
          'доставка осуществлялась' AS shipping_status
     FROM sql.city AS c
LEFT JOIN sql.shipment AS s ON c.city_id = s.city_id
    WHERE s.ship_id IS NOT NULL
    UNION
   SELECT c.city_name,
          c.state,
          'доставка не осуществлялась' AS shipping_status
     FROM sql.city AS c
LEFT JOIN sql.shipment AS s ON c.city_id = s.city_id
    WHERE s.ship_id IS NULL
 ORDER BY 1,
          2
```

**Решение 1** (переусложненный)


```sql
   SELECT shipment_not_completed.city_name,
          shipment_not_completed.state,
          'доставка не осуществлялась' AS shipment_info
     FROM (
             SELECT city.city_name,
                    city.state,
                    shipment.ship_id
               FROM sql.city AS city
          LEFT JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
           GROUP BY city.city_id,
                    shipment.ship_id
             HAVING shipment.ship_id IS NULL
          ) AS shipment_not_completed
UNION ALL
   SELECT city.city_name,
          city.state,
          'доставка осуществлялась'
     FROM sql.city AS city
     JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
 GROUP BY city.city_id
 ORDER BY city_name,
          state
```

**Эталонное решение**


```sql
   SELECT c.city_name AS city_name,
          c.state AS state,
          'доставка осуществлялась' AS shipping_status
     FROM sql.city c
LEFT JOIN sql.shipment s ON c.city_id = s.city_id
    WHERE s.city_id IS NOT NULL
    UNION
   SELECT c.city_name AS city_name,
          c.state AS state,
          'доставка не осуществлялась' AS shipping_status
     FROM sql.city c
LEFT JOIN sql.shipment s ON c.city_id = s.city_id
    WHERE s.city_id IS NULL
 ORDER BY 1,
          2
```

**Решение 2** (оптимизированный эталон)

```sql
   SELECT city.city_name AS city_name,
          city.state AS state,
          'доставка осуществлялась' AS shipping_status
     FROM sql.city AS city
     JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
    UNION
   SELECT city.city_name,
          city.state,
          'доставка не осуществлялась' AS shipping_status
     FROM sql.city AS city
LEFT JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
    WHERE shipment.city_id IS NULL
 ORDER BY city_name,
          state
```


**Решение 3** (вариант 1 исправленный)

```sql
   SELECT city.city_name,
          city.state,
          'доставка не осуществлялась' AS shipment_info
     FROM sql.city AS city
LEFT JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
    WHERE shipment.ship_id IS NULL
UNION ALL
   SELECT city.city_name,
          city.state,
          'доставка осуществлялась'
     FROM sql.city AS city
     JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
 GROUP BY city.city_id
 ORDER BY city_name,
          state
```

#### Задание 5.2

> Напишите запрос, который выводит два столбца: `city_name` и `shippings_fake`. Выведите города, куда совершались доставки. Пусть первый столбец содержит название города, а второй формируется так:
> 
> * если в городе было более десяти доставок, вывести количество доставок в этот город как есть;
> * иначе — вывести количество доставок, увеличенное на пять.
>
> Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.

**Решение после повторения**

```SQL
   SELECT c.city_name,
          COUNT(s.ship_id) AS shippings_fake
     FROM sql.city AS c
     JOIN sql.shipment AS s ON c.city_id = s.city_id
 GROUP BY c.city_id
   HAVING count(s.ship_id) > 10
UNION ALL
   SELECT c.city_name,
          COUNT(s.ship_id) + 5 AS shippings_fake
     FROM sql.city AS c
     JOIN sql.shipment AS s ON c.city_id = s.city_id
 GROUP BY c.city_id
   HAVING count(s.ship_id) <= 10
 ORDER BY 1 ASC,
          2 DESC
```

**Решение** (моё решение совпадает с эталонным)

```sql
   SELECT city.city_name,
          COUNT(shipment.ship_id) AS shippings_fake
     FROM sql.city AS city
     JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
 GROUP BY city.city_id
   HAVING COUNT(shipment.ship_id) > 10
UNION ALL
   SELECT city.city_name,
          COUNT(shipment.ship_id) + 5 AS shippings_fake
     FROM sql.city AS city
     JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
 GROUP BY city.city_id
   HAVING COUNT(shipment.ship_id) <= 10
 ORDER BY shippings_fake DESC,
          city_name
```

### (6) UNION и ручная генерация
---

`UNION` можно использовать для создания справочников прямо в коде запроса. К примеру, если мы хотим вручную ввести какие-то значения и произвести с ними некоторые манипуляции или дополнить существующую выдачу своими значениями.

Составим запрос, который позволит вывести первые три буквы алфавита и их порядковые номера.


```sql
   SELECT 'a' letter,
          '1' ordinal_position
    UNION
   SELECT 'b',
          '2'
    UNION
   SELECT 'c',
          '3'
```

Существуют сложные алгоритмы сравнения текстовых значений, но главный смысл сводится к одному: сравнение производится на основе таблицы `unicode` и позиции элемента в ней с учётом определённых условий.

#### Задание 6.1

> Напишите запрос, который выберет наибольшее из значений:
> 
> * 1000000;
> * 541;
> * -500;
> * 100.
> 
> Столбец с результатом назовите `result`.

**Решение**


```SQL
   SELECT 1000000 AS result
UNION ALL
   SELECT 541
UNION ALL
   SELECT -500
UNION ALL
   SELECT 100
 ORDER BY result DESC
    LIMIT 1
```

#### Задание 6.2

>
> Помним, что сортировка для числовых и строковых типов данных отличается. Построив запрос по аналогии с примером, приведите значения к текстовому типу данных, сравните и выберите наибольшее из них:
> 
> * 1000000;
> * 541;
> * -500;
> * 100.
> 
> Столбец с ответом назовите `mycol`
>

**Решение**


```SQL
   SELECT '1000000' AS val
    UNION
   SELECT '541'
    UNION
   SELECT '-500'
    UNION
   SELECT '100'
 ORDER BY val DESC
    LIMIT 1
```

**Эталонное решение**


```sql
   SELECT '1000000' AS mycol
UNION ALL
   SELECT '541'
UNION ALL
   SELECT '-500'
UNION ALL
   SELECT '100'
 ORDER BY 1 DESC
    LIMIT 1
```

#### Задание 6.3

> Построив запрос по аналогии с примером, найдите самое большое значение из перечисленных операторов:
> 
> * `+` ;
> * `-` ;
> * `=` ;
> * `/` .
> 
> Столбец с ответом назовите `result`.

**Решение**


```sql
   SELECT '+' AS result
UNION ALL
   SELECT '-'
UNION ALL
   SELECT '='
UNION ALL
   SELECT '/'
 ORDER BY result DESC
    LIMIT 1
```

## (7) EXCEPT
---

**Исключаем повторяющиеся данные**

Предположим, нам нужно узнать, в какие города осуществлялась доставка, за исключением тех, в которых проживают водители.


```SQL
   SELECT c.city_name
     FROM sql.shipment s
     JOIN sql.city c ON s.city_id = c.city_id
   EXCEPT
   SELECT cc.city_name
     FROM sql.driver d
     JOIN sql.city cc ON d.city_id = cc.city_id
 ORDER BY 1
```


Все водители проживают в городе `Memphis`, и мы видим, что он не выводится в результате запроса.

Как вы, должно быть, заметили, для решения этой задачи мы использовали оператор `EXCEPT`. Принципы его работы мы сейчас разберём

Чтобы лучше понять данный тип присоединения, предлагаем ознакомиться с **диаграммой Венна** — математическим инструментом, представляющим возможные логические связи между соединёнными наборами данных.

![](data/dst3-u2-md4_7_1.png)

Синтаксические правила для оператора `EXCEPT` такие же, как и для `UNION`:

* одинаковый тип данных;
* одинаковое количество столбцов;
* одинаковый порядок столбцов согласно типу данных.

Синтаксис выглядит следующим образом:


```sql
   SELECT n columns
     FROM table_1
   EXCEPT
   SELECT n columns
     FROM table_2
```

Мы уже знаем, как решить такую задачу с использованием `LEFT` `JOIN`. Вариант с `EXCEPT` будет полезен в тех случаях, когда у вас много столбцов и вам не хочется прописывать их равенство в условии для `JOIN`.

Предположим, у нас есть информация о продажах канцелярского магазина за май и июнь.

Какие-то позиции продавались и в том, и в другом месяце, а какие-то — только в одном. Использовав `EXCEPT`, мы можем оставить только те товары, которые есть в первом запросе (например, за май), но отсутствуют во втором запросе (например, за июнь).

![](data/dst3-u2-md4_7_2.png)

Таким образом, при присоединении с помощью `EXCEPT` мы вывели только те товары, которые были проданы в мае, но не в июне. Чтобы найти продажи по тем позициям, что были реализованы в июне, а в мае — нет, необходимо поменять запросы местами.

Сообщение ментору от 18.09.2023

@Илья_Маяковский_Ментор 

Здравствуйте!

Вопрос по юниту 7 `EXCEPT`

В юните приведена диаграма Венна (скрин) и ниже абзац:

> Мы уже знаем, как решить такую задачу с использованием LEFT JOIN. Вариант с EXCEPT будет полезен в тех случаях, когда у вас много столбцов и вам не хочется прописывать их равенство в условии для JOIN.

Вероятно, я что-то не так понял... Но как решать такую задачу с использованием `LEFT JOIN`?

Решение примера с применением `LEFT JOIN` 


```SQL
   SELECT may.product,
          may.price
     FROM (
             SELECT 'Ручка' AS product,
                    10 AS price
          UNION ALL
             SELECT 'Блокнот',
                    30
          UNION ALL
             SELECT 'Тетрадь',
                    5
          ) AS may
LEFT JOIN (
             SELECT june.product,
                    june.price
               FROM (
                       SELECT 'Тетрадь' AS product,
                              5 AS price
                    UNION ALL
                       SELECT 'Пенал',
                              50
                    UNION ALL
                       SELECT 'Книга',
                              100
                    ) AS june
          ) AS june ON may.product = june.product
    WHERE june IS NULL   
```

**Решение с использованием `EXCEPT`

```SQL
   SELECT may.product,
          may.price
     FROM (
             SELECT 'Ручка' AS product,
                    10 AS price
          UNION ALL
             SELECT 'Блокнот',
                    30
          UNION ALL
             SELECT 'Тетрадь',
                    5
          ) AS may
   EXCEPT
   SELECT june.product,
          june.price
     FROM (
             SELECT 'Тетрадь' AS product,
                    5 AS price
          UNION ALL
             SELECT 'Пенал',
                    50
          UNION ALL
             SELECT 'Книга',
                    100
          ) AS june

```

### Задание 7.1

> Выведите список zip-кодов, которые есть в таблице `sql.driver`, но отсутствуют в таблице `sql.customer`. Отсортируйте по возрастанию, столбец к выводу — `zip`. В поле ниже введите запрос, с помощью которого вы решили эту задачу.

**Решение после повторения**

```SQL
   SELECT d.zip_code AS zip
     FROM sql.driver AS d
   EXCEPT
   SELECT c.zip
     FROM sql.customer AS c
 ORDER BY 1
```

**Решение**


```sql
   SELECT driver.zip_code AS zip
     FROM sql.driver AS driver
   EXCEPT
   SELECT customer.zip
     FROM sql.customer AS customer
 ORDER BY 1
```

**Эталонное решение**

```SQL
   SELECT zip_code AS zip
     FROM sql.driver
   EXCEPT
   SELECT zip
     FROM sql.customer
 ORDER BY 1
 ```

## (8) INTERSECT
---

**Выбираем общие данные**

А что если нам надо вывести общие записи — те, что существуют в нескольких таблицах?

Предположим, нам надо вывести совпадающие по названию города и штаты.


```sql
   SELECT c.city_name object_name
     FROM sql.city c
INTERSECT
   SELECT cc.state
     FROM sql.city cc
 ORDER BY 1
```

Как видим, с помощью оператора `INTERSECT` мы вывели названия городов и штатов, которые совпадают: *New York*, *Washington* и *Wyoming*. Присмотримся к нему внимательнее.

Чтобы лучше понять, как работает этот оператор, вновь обратимся к диаграмме Венна: `INTERSECT` оставляет из результатов первого запроса все строки, которые совпали с результатом выполнения второго запроса.

![](data/dst3-u2-md4_8_1.png)

Синтаксис запроса с оператором `INTERSECT` выглядит следующим образом:


```sql
   SELECT n columns
     FROM table_1
INTERSECT
   SELECT n columns
     FROM table_2
```

Вернёмся к нашему примеру с продажами канцтоваров.

С помощью оператора `INTERSECT` мы можем вывести те позиции, которые продавались и в мае, и в июне. Визуализировать это действие можно примерно так:

![](data/dst3-u2-md4_8_2.png)

Оператор `INTERSECT` оставляет только те строки, которые являются общими для двух запросов (в нашем примере это Тетрадь).

> Как `EXCEPT`, так и `INTERSECT` убирают дубликаты, если они имеются.

### Задание 8.1

> Напишите запрос, который выведет список *id* городов, в которых есть и клиенты, и доставки, и водители.

**Решение после повторения**

```SQL
   SELECT city_id
     FROM sql.customer
INTERSECT
   SELECT city_id
     FROM sql.shipment
INTERSECT
   SELECT city_id
     FROM sql.driver
```

**Решение**


```sql
   SELECT customer.city_id
     FROM sql.customer AS customer
INTERSECT
   SELECT driver.city_id
     FROM sql.driver AS driver
INTERSECT
   SELECT shipment.city_id
     FROM sql.shipment AS shipment
```

### Задание 8.2

> Выведите *zip*-код, который есть как в таблице с клиентами, так и в таблице с водителями.

**Решение после повторения**

```SQL
   SELECT zip
     FROM sql.customer
INTERSECT
   SELECT zip_code
     FROM sql.driver
```

**Решение**


```sql
SELECT
    customer.zip
FROM
    sql.customer AS customer
INTERSECT
SELECT
    driver.zip_code
FROM
    sql.driver AS driver
```

## (9) Итоги. Закрепление знаний

В этом модуле вы освоили ещё несколько возможностей из функционала *SQL*.

Теперь вы умеете:

* присоединять таблицы «снизу», одну под другой;
* исключать или, наоборот, выбирать повторяющиеся записи из двух таблиц;
* вручную добавлять записи в таблицу.

Запишем структуру запроса с учётом полученных знаний.


```sql
SELECT          
    N columns
FROM          
    table_1
UNION / UNION ALL / EXCEPT / INTERSECT 
SELECT          
    N columns
FROM          
    table_2
```

### Задание 9.1

In [ ]:
   SELECT c.city_name,
          s.weight
     FROM sql.city AS c
     JOIN sql.shipment AS s ON c.city_id = s.city_id
    WHERE weight = (
             SELECT MAX(weight)
               FROM sql.shipment
          )
       OR weight = (
             SELECT MIN(weight)
               FROM sql.shipment
          )
 ORDER BY 2 DESC

> Выведите города с максимальным и минимальным весом единичной доставки. Столбцы к выводу — `city_name`, `weight`.

**Решение после повторения (совпадает с эталонным)**

```SQL
(
   SELECT c.city_name,
          s.weight
     FROM sql.city AS c
     JOIN sql.shipment AS s ON c.city_id = s.city_id
 ORDER BY 2 DESC
    LIMIT 1
)
UNION ALL
(
   SELECT c.city_name,
          s.weight
     FROM sql.city AS c
     JOIN sql.shipment AS s ON c.city_id = s.city_id
 ORDER BY 2 ASC
    LIMIT 1
)
```

**Решение после повторения (вариант 2) идея моя - реализация ИИ**

```SQL
     WITH cityweights AS (
             SELECT c.city_name,
                    s.weight
               FROM sql.city AS c
               JOIN sql.shipment AS s ON c.city_id = s.city_id
          )
   SELECT *
     FROM cityweights
    WHERE weight = (SELECT MAX(weight) FROM cityweights)
       OR weight = (SELECT MIN(weight) FROM cityweights)
```

**Усовершенстованное решение без UNION - через фильтрацию (после повторения)**

```SQL
    SELECT c.city_name, s.weight
    FROM sql.city AS c
    JOIN sql.shipment AS s ON c.city_id = s.city_id
WHERE weight = (SELECT MAX(weight) FROM sql.shipment)
   OR weight = (SELECT MIN(weight) FROM sql.shipment)
   ORDER BY 2 DESC
```

**Решение с UNION (после повторения)**


```SQL
    WITH cityweights AS (
             SELECT c.city_name,
                    s.weight
               FROM sql.city AS c
               JOIN sql.shipment AS s ON c.city_id = s.city_id
          )
          --
--
(
    SELECT *
      FROM cityweights
  ORDER BY 2 DESC
     LIMIT 1
)
UNION ALL
(
   SELECT *
     FROM cityweights
 ORDER BY 2 ASC
    LIMIT 1
)
```

**Решение (тупое)**


```sql
(
   SELECT city.city_name AS city_name,
          MAX(shipment.weight) AS weight
     FROM sql.shipment AS shipment
     JOIN sql.city AS city ON shipment.city_id = city.city_id
 GROUP BY city.city_id
 ORDER BY MAX(shipment.weight) DESC
    LIMIT 1
)
UNION ALL
(
   SELECT city.city_name AS city_name,
          MIN(shipment.weight) AS weight
     FROM sql.shipment AS shipment
     JOIN sql.city AS city ON shipment.city_id = city.city_id
 GROUP BY city.city_id
 ORDER BY MIN(shipment.weight) ASC
    LIMIT 1
)
```

### Задание 9.2

> Выведите идентификационные номера клиентов (`cust_id`), которые совпадают с идентификационными номерами доставок (`ship_id`). Столбец к выводу — `mutual_id`. Отсортируйте по возрастанию.

**Решение (совпадает с решением после повторения и с эталонным)**


```sql
   SELECT cust_id AS mutual_id
     FROM sql.customer
INTERSECT
   SELECT ship_id
     FROM sql.shipment
 ORDER BY 1
```

### Задание 9.3

> Создайте справочник, содержащий уникальные имена клиентов, которые являются производителями (`cust_type='manufacturer'`), и производителей грузовиков, а также описание объекта — 'КЛИЕНТ' или 'ГРУЗОВИК'. Столбцы к выводу — `object_name`, `object_description`. Отсортируйте по названию в алфавитном порядке.

**Решение после повторения**

```SQL
   SELECT cust_name AS object_name,
          'КЛИЕНТ' AS object_description
     FROM sql.customer
    WHERE cust_type = 'manufacturer'
    UNION
   SELECT make,
          'ГРУЗОВИК'
     FROM sql.truck
 ORDER BY 1
```

**Решение**


```sql
   SELECT customer.cust_name AS object_name,
          'КЛИЕНТ' AS object_description
     FROM sql.customer AS customer
    WHERE cust_type = 'manufacturer'
    UNION
   SELECT truck.make,
          'ГРУЗОВИК'
     FROM sql.truck AS truck
 ORDER BY object_name
```

In [ ]:
     WITH rnk_books AS (
             SELECT order_month,
                    book_name,
                    cnt,
                    dense_rank() OVER (
                    PARTITION BY order_month
                     ORDER BY cnt DESC,
                              book_average_rating DESC
                    ) rnk
               FROM (
                       SELECT extract(
                              MONTH
                                   FROM order_date
                              ) order_month,
                              b.book_name,
                              b.book_average_rating,
                              count(*) cnt
                         FROM other."book_orders" o
                         JOIN other.books b ON o.book_id = b.book_id
                        WHERE b.publishing_year >= 2021 - 10
                          AND order_date >= to_date('2019-01-01', 'YYYY-MM-DD')
                     GROUP BY extract(
                              MONTH
                                   FROM order_date
                              ),
                              b.book_name,
                              book_average_rating
                    ) month_books
          )
   SELECT *
     FROM rnk_books
    WHERE rnk <= 5
 ORDER BY order_month DESC,
          rnk


In [ ]:
     WITH monthly_orders AS (
             SELECT DATE_PART('month', order_date) AS order_month,
                    b.book_name,
                    count(o.book_id) AS cnt,
                    sum(cnt) AS total_orders,
                    b.book_average_rating
               FROM other.book_orders AS o
               JOIN other.books AS b ON o.book_id = b.book_id
              WHERE b.publishing_year >= 2011
                AND o.order_date >= '2019-01-01'
           GROUP BY order_month,
                    b.book_name,
                    b.book_average_rating
          ),
          ranked_books AS (
             SELECT order_month,
                    book_name,
                    cnt,
                    book_average_rating,
                    dense_rank() OVER (
                    PARTITION BY order_month
                     ORDER BY cnt DESC,
                              book_average_rating DESC
                    ) AS rnk,
                    cnt / total_orders * 100 AS total_ratio,
               FROM monthly_orders
          )
   SELECT order_month,
          book_name,
          cnt,
          rnk,
          total_orders,
          total_ratio
     FROM ranked_books
    WHERE rnk <= 5
 ORDER BY order_month DESC,
          rnk
;


In [ ]:
     WITH monthly_orders AS (
             SELECT DATE_PART('month', order_date) AS order_month,
                    b.book_name,
                    count(o.book_id) AS cnt,
                    b.book_average_rating
               FROM other.book_orders AS o
               JOIN other.books AS b ON o.book_id = b.book_id
              WHERE b.publishing_year >= 2011
                AND o.order_date >= '2019-01-01'
           GROUP BY order_month,
                    b.book_name,
                    b.book_average_rating
          ),
          ranked_books AS (
             SELECT order_month,
                    book_name,
                    cnt,
                    book_average_rating,
                    dense_rank() OVER (
                    PARTITION BY order_month
                     ORDER BY cnt DESC,
                              book_average_rating DESC
                    ) AS rnk
               FROM monthly_orders
          ),
          month_total AS (
             SELECT DATE_PART('month', order_date) AS order_month,
                    count(*) AS total_orders
               FROM other.book_orders AS o
               JOIN other.books AS b ON o.book_id = b.book_id
              WHERE b.publishing_year >= 2011
                AND o.order_date >= '2019-01-01'
           GROUP BY order_month
          )
   SELECT r.order_month,
          book_name,
          cnt,
          rnk,
          total_orders,
          CAST(
          ROUND((CAST(cnt AS NUMERIC) / total_orders * 100), 0) AS INT
          ) AS total_ratio
     FROM ranked_books AS r
     JOIN month_total AS m ON r.order_month = m.order_month
    WHERE rnk <= 5
 ORDER BY order_month DESC,
          rnk
;


In [ ]:
     WITH monthly_orders AS (
             SELECT DATE_PART('month', order_date) AS order_month,
                    b.book_name,
                    COUNT(o.book_id) AS cnt,
                    b.book_average_rating
               FROM other.book_orders AS o
               JOIN other.books AS b ON o.book_id = b.book_id
              WHERE b.publishing_year >= 2011
                AND o.order_date >= '2019-01-01'
           GROUP BY order_month,
                    b.book_name,
                    b.book_average_rating
          ),
          ranked_books AS (
             SELECT order_month,
                    book_name,
                    cnt,
                    book_average_rating,
                    DENSE_RANK() OVER (
                    PARTITION BY order_month
                     ORDER BY cnt DESC,
                              book_average_rating DESC
                    ) AS rnk
               FROM monthly_orders
          ),
          month_total AS (
             SELECT DATE_PART('month', order_date) AS order_month,
                    COUNT(*) AS total_orders
               FROM other.book_orders AS o
               JOIN other.books AS b ON o.book_id = b.book_id
              WHERE b.publishing_year >= 2011
                AND o.order_date >= '2019-01-01'
           GROUP BY order_month
          )
   SELECT r.order_month,
          book_name,
          cnt,
          rnk,
          total_orders,
          FLOOR(CAST(cnt AS NUMERIC) / total_orders * 100)::INTEGER AS total_ratio_floor
     FROM ranked_books AS r
     JOIN month_total AS m ON r.order_month = m.order_month
    WHERE rnk <= 5
 ORDER BY order_month DESC,
          rnk
;

Посчитайте долю продаж каждой книги из выборки от общих продаж книг из топ-5. Столбцы к выводу — order_month, book_name, cnt, rnk, total_orders5, total_ratio5. Примечание. Долю нужно посчитать в процентах — не забудьте умножить на 100! Задачу можно решить с помощью агрегатной оконной функции.

In [ ]:
     WITH monthly_orders AS (
             SELECT DATE_PART('month', order_date) AS order_month,
                    b.book_name,
                    COUNT(o.book_id) AS cnt,
                    b.book_average_rating
               FROM other.book_orders AS o
               JOIN other.books AS b ON o.book_id = b.book_id
              WHERE b.publishing_year >= 2011
                AND o.order_date >= '2019-01-01'
           GROUP BY order_month,
                    b.book_name,
                    b.book_average_rating
          ),
          ranked_books AS (
             SELECT order_month,
                    book_name,
                    cnt,
                    book_average_rating,
                    DENSE_RANK() OVER (
                    PARTITION BY order_month
                     ORDER BY cnt DESC,
                              book_average_rating DESC
                    ) AS rnk
               FROM monthly_orders
          ),
          month_total5 AS (
             SELECT order_month,
                    sum(cnt) AS total_orders5
               FROM ranked_books
              WHERE rnk <= 5
           GROUP BY order_month
          )
   SELECT r.order_month,
          book_name,
          cnt,
          rnk,
          total_orders5,
          (CAST(cnt AS NUMERIC) / NULLIF(total_orders5, 0)) * 100 AS total_ratio5
     FROM ranked_books AS r
     JOIN month_total5 AS m ON r.order_month = m.order_month
    WHERE rnk <= 5
 ORDER BY order_month DESC,
          rnk
;

Посчитайте процент заказов каждой книги из выборки в месяц от общих заказов книг из топа за все месяцы. Столбцы к выводу — order_month, book_name, cnt, rnk, total_orders_all, total_ratio5_all. Примечание. Долю нужно посчитать в процентах — не забудьте умножить на 100!

In [ ]:
     WITH monthly_orders AS (
             SELECT DATE_PART('month', order_date) AS order_month,
                    b.book_name,
                    o.book_id,
                    COUNT(o.book_id) AS cnt,
                    b.book_average_rating
               FROM other.book_orders AS o
               JOIN other.books AS b ON o.book_id = b.book_id
              WHERE b.publishing_year >= 2011
                AND o.order_date >= '2019-01-01'
           GROUP BY order_month,
                    b.book_name,
                    o.book_id,
                    b.book_average_rating
          ),
          ranked_books AS (
             SELECT order_month,
                    book_name,
                    book_id,
                    cnt,
                    book_average_rating,
                    DENSE_RANK() OVER (
                    PARTITION BY order_month
                     ORDER BY cnt DESC,
                              book_average_rating DESC
                    ) AS rnk
               FROM monthly_orders
          ),
          ranked_book_orders_all AS (
             SELECT SUM(cnt)::FLOAT AS total_orders_all
               FROM ranked_books
              WHERE rnk <= 5
          )
   SELECT rb.order_month,
          rb.book_name,
          rb.cnt,
          rb.rnk,
          rbo.total_orders_all,
          (rb.cnt / rbo.total_orders_all) * 100 AS total_ratio_all
     FROM ranked_books rb
    CROSS JOIN ranked_book_orders_all rbo -- Используем CROSS JOIN для распространения значения
    WHERE rb.rnk <= 5
 ORDER BY rb.order_month DESC,
          rb.rnk
;

In [ ]:
     WITH monthly_orders AS (
             SELECT DATE_PART('month', order_date) AS order_month,
                    b.book_name,
                    o.book_id,
                    COUNT(o.book_id) AS cnt,
                    b.book_average_rating
               FROM other.book_orders AS o
               JOIN other.books AS b ON o.book_id = b.book_id
              WHERE b.publishing_year >= 2011
                AND o.order_date >= '2019-01-01'
           GROUP BY order_month,
                    b.book_name,
                    o.book_id,
                    b.book_average_rating
          ),
          ranked_books AS (
             SELECT order_month,
                    book_name,
                    book_id,
                    cnt,
                    book_average_rating,
                    DENSE_RANK() OVER (
                    PARTITION BY order_month
                     ORDER BY cnt DESC,
                              book_average_rating DESC
                    ) AS rnk
               FROM monthly_orders
          ),
          ranked_book_orders_all AS (
             SELECT SUM(cnt)::FLOAT AS total_orders_all
               FROM ranked_books
              WHERE rnk <= 5
          )
   SELECT rb.order_month,
          rb.book_name,
          rb.cnt,
          rb.rnk,
          rbo.total_orders_all,
          (rb.cnt * 100 / rbo.total_orders_all)  AS total_ratio_all
     FROM ranked_books rb
    CROSS JOIN ranked_book_orders_all rbo -- Используем CROSS JOIN для распространения значения
    WHERE rb.rnk <= 5
 ORDER BY rb.order_month DESC,
          rb.rnk
;

In [ ]:
     WITH rnk_books AS (
             SELECT order_month,
                    book_name,
                    cnt,
                    dense_rank() OVER (
                    PARTITION BY order_month
                     ORDER BY cnt DESC,
                              book_average_rating DESC
                    ) rnk
               FROM (
                       SELECT extract(
                              MONTH
                                   FROM order_date
                              ) order_month,
                              b.book_name,
                              b.book_average_rating,
                              count(*) cnt
                         FROM other."book_orders" o
                         JOIN other.books b ON o.book_id = b.book_id
                        WHERE b.publishing_year >= 2021 - 10
                          AND order_date >= to_date('2019-01-01', 'YYYY-MM-DD')
                     GROUP BY extract(
                              MONTH
                                   FROM order_date
                              ),
                              b.book_name,
                              b.book_average_rating
                    ) month_books
          )
   SELECT rnk_books.*,
          sum(cnt) OVER () total_orders_all,
          cnt * 100 / sum(cnt) OVER () total_ratio_all
     FROM rnk_books
    WHERE rnk <= 5
 ORDER BY order_month DESC,
          rnk


In [2]:
2 / 97 * 100

2.0618556701030926

order_month    book_name                                                   cnt    rnk    total_orders_all    total_ratio_all       
-------------------------------------------------------------------------------------------------------------------------------
4.0            Maybe Someday                                               3      1      97.0                3.0927835051546393    
4.0            The Coincidence of Callie & Kayden                          3      2      97.0                3.0927835051546393    
4.0            Queen of Shadows                                            2      3      97.0                2.061855670103093     
4.0            Aristotle and Dante Discover the Secrets of the Universe    2      4      97.0                2.061855670103093     
4.0            Point of Retreat                                            2      5      97.0                2.061855670103093     
3.0            Maybe Someday                                               7      1      97.0                7.216494845360825     
3.0            Beautiful Bastard                                           5      2      97.0                5.154639175257732     
3.0            The Marriage Bargain                                        5      3      97.0                5.154639175257732     
3.0            Seriously...I'm Kidding                                     5      4      97.0                5.154639175257732     
3.0            Onyx                                                        4      5      97.0                4.123711340206186     
2.0            Ugly Love                                                   7      1      97.0                7.216494845360825     
2.0            Maybe Someday                                               6      2      97.0                6.185567010309279     
2.0            The Alloy of Law                                            6      3      97.0                6.185567010309279     
2.0            Between Shades of Gray                                      5      4      97.0                5.154639175257732     
2.0            The Coincidence of Callie & Kayden                          5      5      97.0                5.154639175257732     
1.0            Maybe Someday                                               6      1      97.0                6.185567010309279     
1.0            Ugly Love                                                   6      2      97.0                6.185567010309279     
1.0            Gabriel's Rapture                                           6      3      97.0                6.185567010309279     
1.0            Unearthly                                                   6      4      97.0                6.185567010309279     
1.0            Det som inte dÃ¶dar oss                                     6      5      97.0                6.185567010309279 

order_month    book_name                                                   cnt    rnk    total_orders_all    total_ratio_all       
-------------------------------------------------------------------------------------------------------------------------------
4.0            Maybe Someday                                               3      1      97.0                3.0927835051546393    
4.0            The Coincidence of Callie & Kayden                          3      2      97.0                3.0927835051546393    
4.0            Queen of Shadows                                            2      3      97.0                2.0618556701030926    
4.0            Aristotle and Dante Discover the Secrets of the Universe    2      4      97.0                2.0618556701030926    
4.0            Point of Retreat                                            2      5      97.0                2.0618556701030926    
3.0            Maybe Someday                                               7      1      97.0                7.216494845360824     
3.0            Beautiful Bastard                                           5      2      97.0                5.154639175257731     
3.0            The Marriage Bargain                                        5      3      97.0                5.154639175257731     
3.0            Seriously...I'm Kidding                                     5      4      97.0                5.154639175257731     
3.0            Onyx                                                        4      5      97.0                4.123711340206185     
2.0            Ugly Love                                                   7      1      97.0                7.216494845360824     
2.0            Maybe Someday                                               6      2      97.0                6.185567010309279     
2.0            The Alloy of Law                                            6      3      97.0                6.185567010309279     
2.0            Between Shades of Gray                                      5      4      97.0                5.154639175257731     
2.0            The Coincidence of Callie & Kayden                          5      5      97.0                5.154639175257731     
1.0            Maybe Someday                                               6      1      97.0                6.185567010309279     
1.0            Ugly Love                                                   6      2      97.0                6.185567010309279     
1.0            Gabriel's Rapture                                           6      3      97.0                6.185567010309279     
1.0            Unearthly                                                   6      4      97.0                6.185567010309279     
1.0            Det som inte dÃ¶dar oss                                     6      5      97.0                6.185567010309279